In [22]:
import torch
import pandas as pd
import time
import warnings
import json
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.cuda.amp import autocast

# ---------- Путь к локальной модели DeepSeek ----------
MODEL_PATH = '/home/skoltsov/deepseek_1_5/deepseek_1_5_model/'

# ---------- Параметры инференса ----------
MAX_NEW_TOKENS = 3000
BATCH_SIZE = 62
test_percentage = 1
MAX_LENGTH = 2000

# ---------- Игнорирование предупреждений ----------
warnings.filterwarnings("ignore")

# ---------- Оптимизация скорости ----------
torch.backends.cudnn.benchmark = True

# ---------- Проверка доступности GPU ----------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA недоступна. Убедитесь, что GPU доступна и PyTorch установлен с поддержкой CUDA.")

print("CUDA доступна:", torch.cuda.is_available())
print("Количество доступных GPU:", torch.cuda.device_count())
print("Текущее устройство:", torch.cuda.current_device())
print("Имя устройства:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ---------- Показ GPU памяти ----------
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"🧠 Использовано GPU памяти: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"🛡 Зарезервировано GPU памяти: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
    else:
        print("CUDA недоступна.")

torch.cuda.empty_cache()

# ---------- Измерение времени загрузки модели и данных ----------
start_load_time = time.time()

# ---------- Загрузка токенизатора и модели ----------
print("🚀 Загружаем токенизатор и модель...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    padding_side="left"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16
).to("cuda")  # Явное перемещение модели на GPU

model.eval()

print(f"Модель загружена на устройство: {model.device}")
print_gpu_memory()

# ---------- Загрузка тестовых данных ----------
print("📚 Загружаем тестовые данные...")
path_file = r'/home/skoltsov/deepseek_1_5/data_balansed/test_x_balans1.xlsx'
test_data = pd.read_excel(path_file)
test_data = test_data.sample(frac=test_percentage, random_state=42).reset_index(drop=True)
print(f'📄 Количество уравнений для решения: {len(test_data)}')

end_load_time = time.time()
load_time = end_load_time - start_load_time
print(f"⏱️ Время загрузки модели и данных: {load_time:.2f} секунд")

# ---------- Функция генерации промптов ----------
def build_prompts(equations):
    prompts = []
    for eq in equations:
        prompt = (
            f'Solve the differential equation: {eq}. '
            f'It is must to be provided the final decision in LaTeX format, final solution must be enclosed in \\boxed{{}}'
        )
        prompts.append(prompt)
    return prompts

# ---------- Функция решения уравнений батчем ----------
def solve_equations_batch(equations_batch, max_new_tokens=MAX_NEW_TOKENS):
    prompts = build_prompts(equations_batch)
    
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Перемещение входных данных на GPU

    start_time = time.time()

    with torch.no_grad(), autocast():  # Автоматическое управление преобразованием в float16
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    inference_time = time.time() - start_time

    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    del inputs, outputs
    torch.cuda.empty_cache()

    return generated_texts, inference_time

# ---------- Основной цикл обработки ----------
output_filename = "/home/skoltsov/deepseek_1_5/inference_results/deepseek_1_5_test_x_ballans_3000t.json"
results = []

num_batches = (len(test_data) + BATCH_SIZE - 1) // BATCH_SIZE
total_equations = len(test_data)

print(f"🔢 Общее количество батчей: {num_batches}")

start_global_time = time.time()
current_equation_idx = 0

batch_times = []  # Для сбора времени обработки каждого батча

for batch_idx, start_idx in enumerate(range(0, total_equations, BATCH_SIZE)):
    end_idx = min(start_idx + BATCH_SIZE, total_equations)
    batch = test_data.iloc[start_idx:end_idx]
    equations_batch = batch["equation"].tolist()

    solutions, inference_time = solve_equations_batch(equations_batch)
    batch_times.append(inference_time)

    for i, (eq, sol) in enumerate(zip(equations_batch, solutions)):
        results.append({
            "equation": eq,
            "true_answer": batch.iloc[i].get('true_answer', ''),
            "generated_answer": sol,
            "type_eq": batch.iloc[i].get('type_eq', '')
        })

        current_equation_idx += 1
        elapsed_time = time.time() - start_global_time
        speed = current_equation_idx / elapsed_time
        eta = (total_equations - current_equation_idx) / speed if speed > 0 else float('inf')

        sys.stdout.write(
            f"\r📈 Решено: {current_equation_idx}/{total_equations} | "
            f"⏳ Время: {elapsed_time:.1f}c | 🚀 Скорость: {speed:.2f} уравнений/с | "
            f"🕐 Осталось: {eta:.1f}c"
        )
        sys.stdout.flush()

    print(f"\n⏱️ Время инференса батча: {inference_time:.2f} секунд")
    print_gpu_memory()
    print("=" * 100)

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

end_global_time = time.time()
total_time = end_global_time - start_global_time

# ---------- Вывод статистики времени ----------
print("\n📊 Статистика времени:")
print(f"⏱️ Общее время выполнения: {total_time:.2f} секунд")
print(f"⏱️ Время загрузки модели и данных: {load_time:.2f} секунд")
print(f"⏱️ Среднее время обработки батча: {sum(batch_times) / len(batch_times):.2f} секунд")
print(f"⏱️ Среднее время обработки одного уравнения: {total_time / total_equations:.4f} секунд")

print("\n✅ Инференс завершен!")

CUDA доступна: True
Количество доступных GPU: 1
Текущее устройство: 0
Имя устройства: NVIDIA A100-SXM4-80GB
🚀 Загружаем токенизатор и модель...
Модель загружена на устройство: cuda:0
🧠 Использовано GPU памяти: 15550.53 MB
🛡 Зарезервировано GPU памяти: 19896.00 MB
📚 Загружаем тестовые данные...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📄 Количество уравнений для решения: 1710
⏱️ Время загрузки модели и данных: 70.21 секунд
🔢 Общее количество батчей: 28
📈 Решено: 62/1710 | ⏳ Время: 196.7c | 🚀 Скорость: 0.32 уравнений/с | 🕐 Осталось: 5229.2cc

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



⏱️ Время инференса батча: 196.27 секунд
🧠 Использовано GPU памяти: 15550.53 MB
🛡 Зарезервировано GPU памяти: 16666.00 MB
📈 Решено: 124/1710 | ⏳ Время: 392.1c | 🚀 Скорость: 0.32 уравнений/с | 🕐 Осталось: 5015.2c

...


⏱️ Время инференса батча: 194.92 секунд
🧠 Использовано GPU памяти: 15550.53 MB
🛡 Зарезервировано GPU памяти: 16666.00 MB
📈 Решено: 1612/1710 | ⏳ Время: 5093.3c | 🚀 Скорость: 0.32 уравнений/с | 🕐 Осталось: 309.6c

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



⏱️ Время инференса батча: 194.40 секунд
🧠 Использовано GPU памяти: 15550.53 MB
🛡 Зарезервировано GPU памяти: 16666.00 MB
📈 Решено: 1674/1710 | ⏳ Время: 5291.0c | 🚀 Скорость: 0.32 уравнений/с | 🕐 Осталось: 113.8c

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



⏱️ Время инференса батча: 197.27 секунд
🧠 Использовано GPU памяти: 15550.53 MB
🛡 Зарезервировано GPU памяти: 16666.00 MB
📈 Решено: 1710/1710 | ⏳ Время: 5420.6c | 🚀 Скорость: 0.32 уравнений/с | 🕐 Осталось: 0.0ccc
⏱️ Время инференса батча: 129.32 секунд
🧠 Использовано GPU памяти: 15550.53 MB
🛡 Зарезервировано GPU памяти: 16666.00 MB

📊 Статистика времени:
⏱️ Общее время выполнения: 5420.89 секунд
⏱️ Время загрузки модели и данных: 70.21 секунд
⏱️ Среднее время обработки батча: 193.35 секунд
⏱️ Среднее время обработки одного уравнения: 3.1701 секунд

✅ Инференс завершен!
